In [1]:
import pandas as pd

Data Load

In [6]:
df = pd.read_csv('./data/1_1086.csv', index_col='num')
df

,1,2,3,4,5,6,bonus
num,,,,,,,
1,10,23,29,33,37,40,16
2,9,13,21,25,32,42,2
3,11,16,19,21,27,31,30
4,14,27,30,31,40,42,2
5,16,24,29,40,41,42,3
...,...,...,...,...,...,...,...
1082,21,26,27,32,34,42,31
1083,3,7,14,15,22,38,17
1084,8,12,13,29,33,42,5


In [7]:
df.index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086],
           dtype='int64', name='num', length=1086)

In [8]:
df.columns

Index(['1', '2', '3', '4', '5', '6', 'bonus'], dtype='object')

Preprocessing

Modeling

Evaluation